In [ ]:
import numpy as np
import concurrent.futures
from tqdm import tqdm
import Sunlimb
import importlib
from scipy.ndimage import map_coordinates
importlib.reload(Sunlimb)
%matplotlib notebook

In [ ]:
def count_value(pix, pix0, data0, delta, rsun_arc, crota, cdelt1, cdelt2, max_x, max_y, euv_map_data, order=1):
    """
    计算极坐标图像像素点处对应的值
    :param pix: 元组，行列像素
    :param pix0: 元组，太阳中心点对应的笛卡尔坐标行列像素
    :param data0: 元组，行列起始处的像素点对应的值
    :param delta: 元组，行列每像素对应的
    :param rsun_arc: 太阳半径的角秒数
    :param crota: 图像的旋转角度
    :param cdelt1: 每像素的横向分辨率
    :param cdelt2: 每像素的纵向分辨率
    :param max_x: 横轴最大像素值
    :param max_y: 纵轴最大像素值
    :param euv_map_data: sunpy.Map.map.data
    :param order: 默认为1，表示双线性插值
    :return: 对应像素点的值
    """
    r = (pix[0] - 0.5) * delta[0] + data0[0] * rsun_arc
    theta = ((pix[1] - 0.5) * delta[1] + data0[1] + crota) / 180 * np.pi

    pix_x = r * np.sin(theta) / cdelt1 + pix0[1]
    pix_y = pix0[0] - r * np.cos(theta) / cdelt2

    if 1 <= pix_x <= max_x and 1 <= pix_y <= max_y:
        coords = np.array([[pix_y - 1, pix_x - 1]]).T
        return pix[0], pix[1], map_coordinates(euv_map_data, coords, order=order)[0]
    else:
        return pix[0], pix[1], np.nan


def to_polar(euv_map, r_range, deg_range, order=1, **kwargs):
    """
    坐标转换
    :param euv_map: 原始sunpy.Map.map对象
    :param r_range: 径向坐标范围
    :param deg_range: 角度范围，北极点算0度，顺时针增加
    :param kwargs: deg_pix, r_pix, 分别代表每像素的角度值、角秒值
    :param order: 默认为1，表示双线性插值
    :return:
    """
    rsun_arc = euv_map.fits_header['RSUN_ARC']
    crota = euv_map.fits_header['CROTA']
    cdelt1 = euv_map.fits_header['CDELT1']
    cdelt2 = euv_map.fits_header['CDELT2']
    max_x = euv_map.fits_header["NAXIS1"]
    max_y = euv_map.fits_header["NAXIS2"]
    euv_map_data = euv_map.data
    
    deg_pix = kwargs.get('deg_pix', cdelt1 / rsun_arc * 180 / np.pi)
    r_pix = kwargs.get('r_pix', cdelt1)

    # 计算太阳中心点的像素
    # dpix1 = euv_map.fits_header['CRVAL1'] / cdelt1
    # dpix2 = euv_map.fits_header['CRVAL2'] / cdelt1
    # rota = crota / 180 * np.pi
    # pix_x0 = euv_map.fits_header['CRPIX1'] - dpix1 * np.cos(rota) - dpix2 * np.sin(rota)
    # pix_y0 = euv_map.fits_header['CRPIX2'] - dpix1 * np.sin(rota) + dpix2 * np.cos(rota)
    pix_x0 = euv_map.fits_header['EUXCEN']
    pix_y0 = max_y - euv_map.fits_header['EUYCEN'] + 1

    shape = (int((r_range[1] - r_range[0]) * rsun_arc / r_pix), int((deg_range[1] - deg_range[0]) / deg_pix))
    deg_arr = np.arange(deg_range[0] + 0.5 * deg_pix, deg_range[0] + shape[1] * deg_pix, deg_pix)
    r_arr = np.arange(r_range[0] + 0.5 * r_pix / rsun_arc, r_range[0] + shape[0] * r_pix / rsun_arc, r_pix / rsun_arc)

    tasks = [((i, j), (pix_y0, pix_x0), (r_range[0], deg_range[0]), (r_pix, deg_pix), rsun_arc, crota, cdelt1, cdelt2,
              max_x, max_y, euv_map_data, order)
             for i in range(1, shape[0] + 1) for j in range(1, shape[1] + 1)]

    polar_data = np.empty(shape=shape)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda p: count_value(*p), tasks), total=len(tasks), desc="Processing"))

    for x, y, value in results:
        polar_data[x - 1, y - 1] = value

    return np.flipud(polar_data), r_arr, deg_arr

In [ ]:
example = Sunlimb.Sun("E:/python/projects/alfven/data/solo_L2_eui-hrieuvopn-image_20220330T043000227_V02.fits", num_azimuth_bins=3600)

In [ ]:
polar_data, r_arr, deg_arr = to_polar(example.euv_map, (0.8, 1.15), (170, 180), deg_pix=0.01)

In [ ]:
fig = Sunlimb.plot_data(polar_data, deg_arr, r_arr, title=example.header['DATE-OBS'])